In [1]:
%load_ext autoreload
%autoreload 2

In [27]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from json import load
import tifffile
from tqdm import tqdm
from skimage.draw import polygon
from matplotlib import rcParams
from glasbey import create_palette

palette = {
    "green": "#558150",
    "beige": "#F1E2C3",
    "brown": "#A7785A",
    "pink": "#F0D6C2",
    "black": "#0E0E0E",
}

rcParams['font.family'] = 'sans-serif'
rcParams['figure.facecolor'] = "#FFFFFF00"
rcParams['axes.facecolor'] = "#FFFFFF00"
rcParams['legend.framealpha'] = 0.2
rcParams['axes.edgecolor'] = palette["black"]
rcParams['axes.labelcolor'] = palette["black"]
rcParams['xtick.color'] = palette["black"]
rcParams['ytick.color'] = palette["black"]
rcParams['text.color'] = palette["black"]
rcParams['axes.titlecolor'] = palette["black"]

s_palette = sns.cubehelix_palette(as_cmap=True)
pal = sns.color_palette("dark")
cpal = sns.cubehelix_palette(start=-0.25, rot=2, as_cmap=True)
g_palette = create_palette(8000)

In [42]:
points = pd.read_csv(r"D:\Tracking\NucleiTracking\data\interim\lightsheet\2024_11_04\recon2\dog_lo3_hi5.csv")
tracked_points = pd.read_csv(r"D:\Tracking\NucleiTracking\data\interim\lightsheet\2024_11_04\recon2\recon_fused_tp_181_ch_0_allspots.csv", skiprows=[1, 2, 3], encoding = "ISO-8859-1")
save_path = Path(r"D:\Tracking\NucleiTracking\data\interim\lightsheet\2024_11_04\recon2")

In [43]:
tracked_points

,LABEL,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,VISIBILITY,MANUAL_SPOT_COLOR
0,ID1132785,1132785,79.0,175.529080,326.0,357.0,234.0,0.0,0,5.0,1,NaN
1,ID1132784,1132784,81.0,176.668700,335.0,437.0,220.0,0.0,0,5.0,1,NaN
2,ID1132787,1132787,80.0,173.581600,330.0,408.0,235.0,0.0,0,5.0,1,NaN
3,ID1132786,1132786,697.0,175.125610,247.0,424.0,57.0,0.0,0,5.0,1,NaN
4,ID1132789,1132789,46.0,170.628660,130.0,218.0,55.0,0.0,0,5.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
397657,ID1530412,1530412,NaN,25.753143,264.0,382.0,273.0,150.0,150,5.0,1,NaN
397658,ID1530411,1530411,10101.0,25.769623,283.0,297.0,244.0,150.0,150,5.0,1,NaN
397659,ID1530410,1530410,9417.0,25.791718,87.0,87.0,165.0,150.0,150,5.0,1,NaN
397660,ID1530409,1530409,NaN,25.793304,228.0,448.0,289.0,150.0,150,5.0,1,NaN


In [44]:
import plotly.express as px

xrange = [points['x'].max(), points['x'].min()]
yrange = [points['y'].max(), points['y'].min()]

fig = px.scatter_3d(points, x='x', y='y', z='z', color='val', range_x=xrange, range_y=yrange, animation_frame="frame")
fig.update_traces(marker=dict(size=5))

fig.write_html(save_path / "dog_lo3_hi5.html")

fig = px.scatter_3d(tracked_points, x='POSITION_X', y='POSITION_Y', z='POSITION_Z', color='TRACK_ID', range_x=[0, 100], range_y=yrange, animation_frame="FRAME", color_continuous_scale=g_palette, range_color=[0, 8000])

# draw axes in proportion to the proportion of their ranges
fig.update_layout(scene=dict(aspectmode='manual', aspectratio=dict(x=1, y=points['y'].max()/100, z=points['z'].max()/100)))

fig.update_traces(marker=dict(size=5, colorscale=g_palette))

fig.write_html(save_path / "tracked_allspots.html")
    